In [ ]:
# Imports
import pandas as pd
import numpy as np
import re
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import FastText
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')

In [ ]:
# Carregamento dos dados
df = pd.read_parquet('dados/train.parquet')
df = df[['user_input', 'uf', 'razaosocial', 'nome_fantasia']].reset_index(drop=True)

def clean_text(text):
    """Pré-processamento melhorado"""
    text = str(text).lower().strip()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    # Preserva números e caracteres especiais importantes
    text = re.sub(r'[^a-z0-9\s&.-]', ' ', text)
    
    # Trata números como tokens especiais
    text = re.sub(r'(\d+)', r' \1 ', text)
    
    # Remove stopwords específicas do domínio
    stopwords = ['ltda', 'me', 'epp', 'sa', 's/a', 'limitada', 'eireli', 'comercio', 'servicos']
    words = text.split()
    words = [w for w in words if w not in stopwords]
    
    # Normaliza espaços
    return ' '.join(words)

In [ ]:
# Criação do campo de texto e do target
df['target_empresa'] = (df.razaosocial.fillna('') + ' ' + df.nome_fantasia.fillna('')).apply(clean_text)

In [ ]:
df_reduzido = df[:10000]

# Separação treino/teste
df_train, df_test = train_test_split(df_reduzido, 
                                     test_size=0.2, 
                                     random_state=42)


# FastText training

In [ ]:
# Função para preparar o texto para busca, priorizando nome fantasia
def prepare_search_text(row):
    razao = clean_text(str(row.razaosocial)) if pd.notna(row.razaosocial) else ''
    fantasia = clean_text(str(row.nome_fantasia)) if pd.notna(row.nome_fantasia) else ''
    
    # Se tiver ambos, combina dando mais peso ao nome fantasia
    if razao and fantasia:
        # Repete o nome fantasia para dar mais peso
        return f"{fantasia} {fantasia} {razao}"
    # Se tiver só um deles, usa o que tiver
    return fantasia or razao

# Aplica o prepare_search_text em todos os DataFrames
df_train['search_text'] = df_train.apply(prepare_search_text, axis=1)
df_test['search_text'] = df_test.apply(prepare_search_text, axis=1)

# Tokenização específica para nomes de empresas
def tokenize_business(text):
    """Tokenização que preserva partes importantes do nome"""
    text = str(text).lower().strip()
    # Remove acentos
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    # Preserva números e caracteres especiais importantes
    text = re.sub(r'[^a-z0-9\s&.-]', ' ', text)  # Corrigido o regex
    
    # Trata números como tokens especiais
    text = re.sub(r'(\d+)', r' \1 ', text)
    
    # Normaliza espaços
    text = re.sub(r'\s+', ' ', text)
    
    tokens = []
    for word in text.split():
        # Se tem número, mantém junto
        if any(c.isdigit() for c in word):
            tokens.append(word)
        else:
            # Tokeniza mas mantém palavras pequenas (possíveis iniciais)
            if len(word) <= 2:
                tokens.append(word)
            else:
                tokens.extend(word_tokenize(word))
    return tokens

# FastText com parâmetros otimizados para nomes de empresas
corpus_ft = [tokenize_business(text) for text in df_train.search_text]

fasttext_model = FastText(
    sentences=corpus_ft,
    vector_size=500,     # Aumentar dimensionalidade
    window=5,            # Aumentar janela para capturar mais contexto
    min_count=1,        # Manter palavras raras
    sg=1,               # Skip-gram
    hs=1,               # Hierarchical softmax
    negative=15,        # Aumentar negative sampling
    epochs=50,         # Aumentar épocas
    min_n=1,           # Subpalavras menores
    max_n=8,           # Subpalavras maiores
    workers=4          # Paralelização
)



def improved_ft_embedding(text):
    """Função melhorada para gerar embeddings com pesos adaptativos"""
    tokens = tokenize_business(text)
    vectors = []
    weights = []
    
    for i, token in enumerate(tokens):
        if token in fasttext_model.wv:
            vec = fasttext_model.wv[token]
            # Pesos adaptativos baseados nas características do token
            weight = 1.0
            
            # Primeiras palavras são mais importantes
            if i < 2:
                weight *= 1.6
            
            # # Tokens com números são importantes (filiais, números de loja)
            # if any(c.isdigit() for c in token):
            #     weight *= 1.3
            
            # Tokens curtos (possíveis iniciais) têm peso menor
            if len(token) <= 2:
                weight *= 0.4
                
            vectors.append(vec)
            weights.append(weight)
    
    if vectors:
        weights = np.array(weights).reshape(-1, 1)
        weighted_vectors = np.array(vectors) * weights
        # Normaliza o embedding final
        emb = np.sum(weighted_vectors, axis=0) / np.sum(weights)
        return emb / np.linalg.norm(emb)
    
    return np.zeros(fasttext_model.vector_size)

# Aplica os novos embeddings
print("Gerando embeddings FastText...")
df_train['ft_emb'] = df_train.search_text.apply(improved_ft_embedding)
df_test['ft_emb'] = df_test.search_text.apply(improved_ft_embedding)


## Avaliar modelo

In [ ]:
def get_topk(model_name, user_input, uf=None, k=5, base_busca=None):
    texto = clean_text(user_input)
    data = base_busca.copy()
    
    if uf:
        data = data[data.uf == uf].reset_index(drop=True)
    
    if model_name == 'fasttext':
        # Usa a função ft_embedding para gerar o embedding do FastText
        emb = improved_ft_embedding(texto)
        sims = cosine_similarity([emb], list(data.ft_emb.values))[0]
    elif model_name == 'tfidf':
        vec = tfidf.transform([texto])
        # Garantimos que estamos usando os índices corretos do DataFrame original
        indices_originais = data.index.values
        sims = cosine_similarity(vec, tfidf_matrix[indices_originais])[0]
    elif model_name == 'bert':
        emb = bert_model.encode([texto])[0]
        sims = cosine_similarity([emb], list(data.bert_emb.values))[0]

    top_indices = sims.argsort()[::-1][:k]
    return data.iloc[top_indices]


In [ ]:
def avaliar_modelo(model_name, df_teste, base_busca, batch_size=32):
    total = len(df_teste)
    top1 = 0
    top5 = 0
    
    # Pré-processamento dos alvos
    alvos = df_teste.target_empresa.apply(clean_text).values
    
    # Processamento em lotes com barra de progresso
    with tqdm(total=total, desc=f'Avaliando {model_name}') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            # Processa cada entrada do lote
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                uf = row.uf if hasattr(row, 'uf') else None
                
                # Obtém os top-k resultados
                resultados = get_topk(model_name, entrada, uf, k=5, base_busca=base_busca)
                pred_empresas = resultados.target_empresa.apply(clean_text).values
                
                # Atualiza métricas usando comparação vetorizada
                if alvos[i+j] == pred_empresas[0]:
                    top1 += 1
                if alvos[i+j] in pred_empresas:
                    top5 += 1
                
                # Atualiza a barra de progresso
                pbar.update(1)
    
    # Calcula e mostra as métricas
    print(f'\nModelo: {model_name}')
    print(f'Top-1: {top1/total:.2%} | Top-5: {top5/total:.2%}')


In [ ]:
avaliar_modelo('fasttext', df_test, df_train)

In [ ]:
get_topk('fasttext', 'rener', uf='SP', base_busca=df_test)

## Ensemble

In [ ]:
def hybrid_similarity(query, target, query_vec, target_vec, uf_match=False):
    """Similaridade híbrida com múltiplos fatores"""
    # Similaridade FastText
    ft_sim = cosine_similarity([query_vec], [target_vec])[0][0]
    
    # Similaridade de caracteres
    char_sim = fuzz.ratio(query, target) / 100.0
    
    # Similaridade de prefixo
    prefix_sim = 1.0 if target.startswith(query) else 0.0
    
    # Similaridade por tokens comuns
    query_tokens = set(query.split())
    target_tokens = set(target.split()) 
    token_sim = len(query_tokens & target_tokens) / len(query_tokens | target_tokens)
    
    # Pesos diferentes baseado no tamanho da query
    if len(query) < 5:
        final_sim = 0.4*ft_sim + 0.3*char_sim + 0.2*prefix_sim + 0.1*token_sim
    else:
        final_sim = 0.5*ft_sim + 0.2*char_sim + 0.1*prefix_sim + 0.2*token_sim
    
    # Boost para match de UF
    if uf_match:
        final_sim *= 1.2
        
    return final_sim

def avaliar_hybrid_similarity(df_teste, base_busca, batch_size=32):
    """Avalia as métricas Top-1 e Top-5 usando hybrid_similarity"""
    print("\nAvaliando modelo Híbrido...")
    total = len(df_teste)
    top1 = 0 
    top5 = 0

    # Pré-processamento dos alvos
    alvos = df_teste.target_empresa.apply(clean_text).values

    with tqdm(total=total, desc='Avaliando Híbrido') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                uf = row.uf if hasattr(row, 'uf') else None
                
                texto = clean_text(entrada)
                query_vec = improved_ft_embedding(texto)
                
                # Calcula similaridades
                similarities = []
                for idx, target_row in base_busca.iterrows():
                    target = clean_text(target_row.target_empresa)
                    target_vec = target_row.ft_emb
                    
                    sim = hybrid_similarity(
                        texto, 
                        target,
                        query_vec, 
                        target_vec,
                        uf_match=(uf == target_row.uf if uf else False)
                    )
                    similarities.append(sim)

                # Obtém top-k mais similares
                top_indices = np.array(similarities).argsort()[::-1][:5]
                pred_empresas = base_busca.iloc[top_indices].target_empresa.apply(clean_text).values
                
                # Atualiza métricas
                if alvos[i+j] == pred_empresas[0]:
                    top1 += 1
                if alvos[i+j] in pred_empresas:
                    top5 += 1
                    
                pbar.update(1)
    
    print(f'\nModelo: Hybrid Similarity')
    print(f'Top-1: {top1/total:.2%} | Top-5: {top5/total:.2%}')
    
    return {'top1': top1/total, 'top5': top5/total}

In [ ]:
resultados = avaliar_hybrid_similarity(df_test, df_train)
print(f"Top-1 Accuracy: {resultados['top1']:.4f}")
print(f"Top-5 Accuracy: {resultados['top5']:.4f}")